In [1]:
# Install the required libraries
from IPython.display import clear_output
try:
  !pip install pystan~=2.14
  !pip install fbprophet
  !pip install hvplot
  !pip install holoviews
except:
  print("Error installing libraries")
finally:
  clear_output()
  print('Libraries successfully installed')

Libraries successfully installed


In [2]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
import datetime as dt
import holoviews as hv
from fbprophet import Prophet

%matplotlib inline

Output hidden; open in https://colab.research.google.com to view.

In [3]:
from google.colab import files
uploaded = files.upload()

Saving AAPL.csv to AAPL.csv
Saving ABBV.csv to ABBV.csv
Saving AMZN.csv to AMZN.csv
Saving BAC.csv to BAC.csv
Saving BRK-B.csv to BRK-B.csv
Saving CVX.csv to CVX.csv
Saving GOOG.csv to GOOG.csv
Saving GOOGL.csv to GOOGL.csv
Saving HD.csv to HD.csv
Saving JNJ.csv to JNJ.csv
Saving JPM.csv to JPM.csv
Saving KO.csv to KO.csv
Saving LLY.csv to LLY.csv
Saving MA.csv to MA.csv
Saving META.csv to META.csv
Saving MRK.csv to MRK.csv
Saving MSFT.csv to MSFT.csv
Saving NVDA.csv to NVDA.csv
Saving PEP.csv to PEP.csv
Saving PFE.csv to PFE.csv
Saving PG.csv to PG.csv
Saving TSLA.csv to TSLA.csv
Saving UNH.csv to UNH.csv
Saving V.csv to V.csv
Saving XOM.csv to XOM.csv


In [4]:
# list of top 25 SP500 stocks 
top_25_sp500_stocks = ['AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL', 
                       'GOOG', 'BRK-B', 'UNH', 'JNJ', 'XOM', 
                       'JPM', 'META', 'V', 'PG', 'NVDA', 'HD',
                       'CVX', 'LLY', 'MA', 'ABBV', 'PFE', 'MRK', 
                       'PEP', 'BAC', 'KO']

In [8]:
for stock in top_25_sp500_stocks: 
  # Import the top 25 sp500 stocks dataset into a Pandas Dataframe
  stocks_df = pd.read_csv(
    (stock + ".csv"), 
    index_col='Date', 
    infer_datetime_format=True, 
    parse_dates=True
  )

  stocks_df = stocks_df.sort_values(by=['Date'], ascending = True)

  # drop un-necessary columns
  stocks_df = stocks_df.loc[:, ~stocks_df.columns.str.contains('^Unnamed')]
  stocks_df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

  stocks_df = stocks_df.dropna()

  # Reset the index so that we recode the columns for Prophet
  stocks_df.reset_index(inplace = True)

  # Label the columns ds and y so that the syntax is recognized by Prophet
  prophet_df = stocks_df.rename(columns = {'Date' : 'ds', 'Adj Close' : 'y'})

  # Sort the DataFrame by `ds` in ascending order
  prophet_df = prophet_df.sort_values('ds', ascending = True)

  # Call the Prophet function, store as an object
  model_stocks = Prophet()

  # Fit the Prophet model.
  model_stocks.fit(prophet_df)

  # Create a future dataframe to hold predictions
  # Make the prediction go out as far as 1000 hours (approx 40 days)
  future_trends = model_stocks.make_future_dataframe(periods = 30, freq = 'D')

  # Make the predictions for the trend data using the future_trends DataFrame
  forecast_trends = model_stocks.predict(future_trends)

  # At this point, it's useful to set the `datetime` index of the forecast data.
  forecast_trends.set_index('ds', inplace = True)

  # From the `forecast_trends` DataFrame, use hvPlot to visualize
  #  the yhat, yhat_lower, and yhat_upper columns over the last 10 days (24*10 = 240) 
  # Set `bokeh` to render hvPlot charts
  hv.extension("bokeh")

  forecast_trends = forecast_trends.reset_index()
  forecast_trends_updated = forecast_trends[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

  # Create a `forecast_march_2021` Dataframe, which contains just forecasts for that month
  # The DataFrame should include the columns yhat_upper, yhat_lower, and yhat
  forecast = forecast_trends[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

  # Replace the column names to something less technical sounding
  forecast = forecast.rename(columns = 
                                             {'yhat' : 'Most Likely Case',
                                              'yhat_lower' : 'Worst Case',
                                              'yhat_upper' : 'Best Case'}
                                             )

  forecast = forecast.loc['2022-12-19':]

  forecast.to_csv(stock + '_forecast.csv', index = False)


Output hidden; open in https://colab.research.google.com to view.